In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from n_dimensional_datasets import *
from plotter import *

# approximation methods
from sklearn.ensemble import RandomForestRegressor
from scipy.interpolate import LinearNDInterpolator

In [5]:
start = -1
stop = 1
n_samples = 1000
n_test_samples = n_samples

In [6]:
n_features = 5

sim_data = {"start": start, "stop": stop, "samples": n_samples, "test samples": n_test_samples, "seed": []}
stats = {"score": [], "error": [], "scorevar": []}
approx_type = {"rf": copy.deepcopy(stats), "interp": copy.deepcopy(stats)}

seeds = range(2)
for seed in seeds:
    # create data
    X = stochastic_mesh(n_features, start, stop, n_samples, seed=seed).T
    X_test = stochastic_mesh(n_features, start, stop, n_test_samples, seed=seed).T
    y = decaying_sinewave_nd(X.T, 0.1)
    y_test = decaying_sinewave_nd(X_test.T, 0.1)
    
    # setup approximators
    forest = RandomForestRegressor()
    forest.fit(X,y)
    
    interp = LinearNDInterpolator(X, y, fill_value=0)
    
    # score approximators
    forest_xval_score = cross_val_score(forest, X, y, cv=5)
    approx_type["rf"]["error"].append(np.mean(forest.predict(X)-y))
    approx_type["rf"]["score"].append(forest_xval_score.mean())
    approx_type["rf"]["scorevar"].append(forest_xval_score.std())
    
    interp_y_hat = interp(X_test)
    interp_score = r2_score(y, interp_y_hat)
    approx_type["interp"]["error"].append(np.mean(interp_y_hat - y))
    approx_type["interp"]["score"].append(interp_score)
    approx_type["interp"]["scorevar"].append(np.nan)
    
    sim_data["seed"].append(seed)

for t in approx_type:
    print(t + " Stats: {}".format(pd.DataFrame(approx_type[t])))

rf Stats:       score     error  scorevar
0  0.811469  0.001106  0.006667
1  0.833462  0.002147  0.015650
interp Stats:       score         error  scorevar
0  1.000000 -3.093012e-18       NaN
1  0.999545 -1.052906e-04       NaN
